# Data collation and analysis
This file handles collecting all collected data into one consistent table.

## Data Collection
First data is collected, then any non-gaia stellar IDs (such as HD numbers) are matched 
with a corresponding Gaia star entry. 

## Analysis
After data has been collected into a common table, mass analysis can be performed

In [21]:
import pandas as pd
import numpy as np
from astropy.table import Table
from astroquery.gaia import Gaia

Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"  # Select Data Release 3, default

## Log in to Gaia
Use credentials from gaia/CREDENTIALS file

In [22]:
Gaia.login(credentials_file='gaia/CREDENTIALS')
username = 'mwidmaie'

INFO: Login to gaia TAP server [astroquery.gaia.core]
OK
INFO: Login to gaia data server [astroquery.gaia.core]
OK


## Radii Data
The initial radii dataset was insufficient, so we need to use another dataset and combine them into one. 
We will use the data from `data/stellar_ldd.vot` as well as some Gaia data to manually calculate the stellar radii (using trigonometry).

If the stellar_ldd table is available locally, please use the `vizier_ii346.ipynb` notebook to download it from VizieR.

In [4]:
stellar_ldd_table_name = 'user_' + username + ".stellar_ldd"

try:
    stellar_ldd = Gaia.load_table(stellar_ldd_table_name)
except:
    # Table doesn't exist so we're gonna upload from the local stellar_ldd.vot file
    print("Table does not exist, uploading from local machine.")
    Gaia.upload_table(upload_resource="data/stellar_ldd.vot", table_name='stellar_ldd', format="VOTable")
    stellar_ldd = Gaia.load_table(stellar_ldd_table_name)

Retrieving table 'user_mwidmaie.stellar_ldd'
500 Error 500:
esavo.tap.TAPException: Code: 404, msg: Table 'user_mwidmaie.stellar_ldd' not found.
Table does not exist, uploading from local machine.
Sending file: data/stellar_ldd.vot
Uploaded table 'stellar_ldd'.
Retrieving table 'user_mwidmaie.stellar_ldd'


## Update metadata for stellar_ldd
This will make the next steps a lot easier. (and faster)

In [5]:
Gaia.update_user_table(table_name=stellar_ldd_table_name,
                       list_of_changes=[['"RAJ2000"', 'flags', 'Ra'], ['"DEJ2000"', 'flags', 'Dec'],
                                        ['"RAJ2000"', 'indexed', True], ['"DEJ2000"', 'indexed', True]])

Retrieving table 'user_mwidmaie.stellar_ldd'
Table 'user_mwidmaie.stellar_ldd' updated.


## Retrieving Gaia Parallax and Parallax Error
We need to retrieve the parallax and parallax error from Gaia for each star in the stellar_ldd table.

With this data, we can calculate the distance to the star, and then use the angular diameter from the stellar_ldd table to calculate the stellar radius.
We can combine the data using the following ADQL query:
```sql
SELECT TOP 100000 recno, source_id, Name, RAJ2000, DEJ2000, LDD, e_LDD, ra as gaia_ra, dec as gaia_dec, vmag, parallax, parallax_error,
DISTANCE(
    POINT(RAJ2000, DEJ2000),
    POINT(ra, dec)
) * 3600. AS dist_arcsec
FROM user_<username>.stellar_ldd AS stellar_ldd
JOIN gaiadr3.gaia_source AS gaia
-- Geometric Cross-Match: =======
ON DISTANCE(
    POINT(RAJ2000, DEJ2000),
    POINT(ra, dec)
) < 1.8 / 3600.
-- Condition: ===================
WHERE ABS(vmag - phot_g_mean_mag) < 2.
```

Processing will be done in Python.

In [6]:
import gzip

job = Gaia.launch_job(f"""SELECT TOP 100000 recno, source_id, Name, RAJ2000, DEJ2000, ra as gaia_ra, dec as gaia_dec, vmag, parallax, parallax_error, LDD, e_LDD,
DISTANCE(
    POINT(RAJ2000, DEJ2000),
    POINT(ra, dec)
) * 3600. AS dist_arcsec
FROM {stellar_ldd_table_name} AS stellar_ldd
JOIN gaiadr3.gaia_source AS gaia
-- Geometric Cross-Match: =======
ON DISTANCE(
    POINT(RAJ2000, DEJ2000),
    POINT(ra, dec)
) < 1.8 / 3600.
-- Condition: ===================
WHERE ABS(vmag - phot_g_mean_mag) < 2.""", dump_to_file=True, output_format='votable',
                      output_file='data/stellar_ldd_gaia.vot.gz', verbose=True)

results = job.get_results()

with gzip.open('data/stellar_ldd_gaia.vot.gz', 'rb') as f:
    with open('data/stellar_ldd_gaia.vot', 'wb') as f2:
        f2.write(f.read())

Launched query: 'SELECT TOP 100000 recno, source_id, Name, RAJ2000, DEJ2000, ra as gaia_ra, dec as gaia_dec, vmag, parallax, parallax_error, LDD, e_LDD,
DISTANCE(
    POINT(RAJ2000, DEJ2000),
    POINT(ra, dec)
) * 3600. AS dist_arcsec
FROM user_mwidmaie.stellar_ldd AS stellar_ldd
JOIN gaiadr3.gaia_source AS gaia
-- Geometric Cross-Match: =======
ON DISTANCE(
    POINT(RAJ2000, DEJ2000),
    POINT(ra, dec)
) < 1.8 / 3600.
-- Condition: ===================
WHERE ABS(vmag - phot_g_mean_mag) < 2.'
------>https
host = gea.esac.esa.int:443
context = /tap-server/tap/sync
Content-type = application/x-www-form-urlencoded
200 200
[('Date', 'Fri, 19 Jan 2024 16:13:16 GMT'), ('Server', 'Apache/2.4.6 (CentOS) OpenSSL/1.0.2k-fips mod_jk/1.2.43'), ('Cache-Control', 'no-cache, no-store, max-age=0, must-revalidate'), ('Pragma', 'no-cache'), ('Expires', '0'), ('X-XSS-Protection', '1; mode=block'), ('X-Frame-Options', 'SAMEORIGIN'), ('X-Content-Type-Options', 'nosniff'), ('Content-Encoding', 'gzip'), ('

## Calculating Stellar Radii
### Calculating Distance
We can calculate the distance to the star using the parallax from Gaia, using the following formula:
![Distance Formula](imgs/latex_distance.png)
Where:
d = distance to star (in parsecs)
p = parallax angle (in mas)

### Calculating Stellar Radius
We can calculate the stellar radius using the angular diameter from the stellar_ldd table, using the following formula:
![Stellar Radius Formula](imgs/latex_radius.png)
Where:
r_star = stellar radius (in kilometers)
d = distance to star (in parsecs)
theta = angular diameter (in radians)

### Calculating Stellar Radius Error
We can calculate the stellar radius error using the following formula:
![Stellar Radius Error Formula](imgs/latex_radius_error.png)

Where:
Δr = Stellar radius error (in kilometers)
Δd = Distance error (in km), calculated using the following formula:
![Distance Error Formula](imgs/latex_distance_error.png)

Where:
Δd = Distance error (in km)
d = distance to star (in parsecs)
Δp = Parallax error (in mas)
p = parallax angle (in mas)

Δθ = Angular diameter error (in radians), Given
θ = Angular diameter (in radians), Given

In [20]:
df = Table.read('data/stellar_ldd_gaia.vot').to_pandas()

df = df[df['parallax'] > 0]

df['distance'] = 1000 / df['parallax'] * 3.086e+13  # Convert parallax to distance in km

# Convert LDD to radians (First convert to arcseconds, then to degrees, then to radians)
df['LDD_rad'] = np.radians(df['LDD'] / 1000 / 3600)

# Convert e_LDD to radians
df['e_LDD_rad'] = np.radians(df['e_LDD'] / 1000 / 3600)

# Calculate radius in km 
df['radius'] = df['distance'] * np.tan(1/2 * df['LDD_rad'])

# Calculate distance error in parsecs
df['distance_error'] = 1000 / np.square(df['parallax']) * df['parallax_error'] * 3.086e+13

df['radius_error'] = np.sqrt(
    np.square(df['distance_error'] * np.tan(1 / 2 * df['LDD_rad'])) + 
    np.square(df['distance'] * 
             (1 / np.square(np.cos(df['LDD_rad']))) 
              * 1 / 2 * df['e_LDD_rad']
    )
)

df['radius_error_ratio'] = df['radius_error'] / df['radius']  # Calculate radius error as a ratio

df['radius'] = df['radius'] / 695700  # Convert radius to solar radii
df['radius_error'] = df['radius_error'] / 695700  # Convert radius error to solar radii
df['relative_radius_error'] = df['radius_error'] / df['radius']  # Calculate radius error as a ratio

df = df[df['relative_radius_error'] < .05]  # Remove data with radius error greater than 5%

df.drop(columns=['LDD_rad', 'e_LDD_rad'], inplace=True)
df.dropna(subset=['distance', 'distance_error', 'radius', 'radius_error', 'source_id', 'gaia_dec', 'gaia_ra'],
          inplace=True)
df.drop(columns=['RAJ2000', 'DEJ2000', 'Name', 'dist_arcsec', 'vmag', 'recno', 'LDD', 'e_LDD', 'parallax', 'parallax_error', 'relative_radius_error'], inplace=True)

df

,source_id,gaia_ra,gaia_dec,distance,radius,distance_error,radius_error,radius_error_ratio
0,122737281195008,45.196005,0.993298,4.985357e+15,1.867323,1.403324e+13,0.046556,0.024932
1,261756782464128,45.871279,1.512243,4.179486e+15,1.746986,8.740422e+12,0.047993,0.027472
2,332469124171904,44.819887,1.244554,1.550686e+16,3.469091,2.304786e+14,0.108554,0.031292
4,613978460597248,46.935937,1.847510,5.353828e+15,1.982953,1.798111e+13,0.050898,0.025668
5,1102883177746176,46.606522,3.264338,5.005710e+15,2.781914,2.375626e+13,0.067974,0.024434
...,...,...,...,...,...,...,...,...
68653,6917265660248260736,314.118203,-1.569706,5.222316e+15,1.899961,1.472064e+13,0.043637,0.022968
68654,6917321597900574720,313.972545,-1.338387,9.483958e+15,2.465559,4.370412e+13,0.068918,0.027952
68655,6917474670535481728,314.853266,-0.539722,5.254006e+15,1.780285,2.155898e+13,0.044233,0.024846
68656,6917489002841762304,314.741707,-0.423198,7.264611e+15,2.261141,1.272831e+14,0.070881,0.031347


In [23]:
# Save as a VOTable and upload to Gaia
stellar_radii_table_name = 'user_' + username + ".stellar_radii"
stellar_radii_vot = Table.from_pandas(df)
stellar_radii_vot.write('data/stellar_radii_finalized.vot', format='votable', overwrite=True)

Gaia.upload_table(upload_resource='data/stellar_radii_finalized.vot', table_name='stellar_radii', format='votable')
Gaia.update_user_table(table_name=stellar_radii_table_name,
                       list_of_changes=[['gaia_ra', 'flags', 'Ra'], ['gaia_dec', 'flags', 'Dec'],
                                        ['gaia_ra', 'indexed', True], ['gaia_dec', 'indexed', True]])
Gaia.delete_user_table(table_name=stellar_ldd_table_name)  # Delete the old stellar_ldd table

Sending file: data/stellar_radii_finalized.vot
Uploaded table 'stellar_radii'.
Retrieving table 'user_mwidmaie.stellar_radii'
Table 'user_mwidmaie.stellar_radii' updated.
Table 'user_mwidmaie.stellar_ldd' deleted.


## Collating with stellar mass data
Using data from `data/stellar_radii_combined.vot` we can pair up the Gaia source ids from `data/stellar_masses.rawdata` to form a new dataset including both radii and mass, 2 of the 3 data points we need to calculate stellar core temperature

In [12]:
# We need to download and pre-process the data from https://cdsarc.cds.unistra.fr/viz-bin/nph-Cat/txt?I/360/binmass.dat.gz
# This is a table of stellar masses
import requests

url = 'https://cdsarc.cds.unistra.fr/viz-bin/nph-Cat/txt?I/360/binmass.dat'
print("Downloading data from " + url + "... This may take a while.")
r = requests.get(url, allow_redirects=True)

with open('data/binmass.rawdata', 'wb') as f:
    f.write(r.content)

print('Download complete, processing...')
fixed_content = ''
# Now we need to process the data, since it's not in a standard format
with open('data/binmass.rawdata', 'r') as f:
    lines = f.readlines()[4:-1]
    for line in lines:
        if line[0] == '#' or line[0] == '-':
            continue
        content = line.replace('|', ',')
        ra_and_dec = content.split(',')[13]
        ra = ra_and_dec.split(' ')[0]
        dec = ra_and_dec.split(' ')[-1]
        content = content.replace(ra_and_dec, ra + ',' + dec).replace(' ', '')
        fixed_content += content

with open('data/stellar_masses.rawdata', 'w') as f:
    f.write(fixed_content)

print('Done')

Download complete, processing...
Done


## Uploading stellar_masses to Gaia
We need to upload the stellar_masses data to Gaia so we can join it with the stellar_radii data.

In [24]:
# Load stellar_masses.rawdata as a Pandas DataFrame

# Removed the column headers because they weren't specific enough, we'll use these column names instead
columns = ['source_id', 'mass_primary', 'mass_primary_lower', 'mass_primary_upper', 'mass_secondary',
           'mass_secondary_lower', 'mass_secondary_upper', 'flux_ratio', 'flux_ratio_lower',
           'flux_ratio_upper', 'method', 'reference_for_primary_mass', 'flag', 'ra2016', 'dec2016']

# Load the data into a Pandas DataFrame
stellar_masses = pd.read_csv('data/stellar_masses.rawdata', names=columns, skiprows=4)
stellar_masses.drop(
    columns=['mass_secondary_lower', 'mass_secondary_upper', 'flux_ratio_lower', 'flux_ratio_upper', 'flag', 'method',
             'reference_for_primary_mass'], inplace=True)

# Drop any with NaN ra or dec
stellar_masses.dropna(subset=['ra2016', 'dec2016'], inplace=True)

# Save as a VOTable
stellar_masses_vot = Table.from_pandas(stellar_masses)
stellar_masses_vot.write('data/stellar_masses.vot', format='votable', overwrite=True)

# Upload to Gaia
Gaia.upload_table(upload_resource='data/stellar_masses.vot', table_name='stellar_masses', format='votable')

Sending file: data/stellar_masses.vot
Uploaded table 'stellar_masses'.


In [25]:
# Add metadata to the table
stellar_masses_table_name = 'user_' + username + ".stellar_masses"

Gaia.update_user_table(table_name=stellar_masses_table_name,
                       list_of_changes=[['ra2016', 'flags', 'Ra'], ['dec2016', 'flags', 'Dec'],
                                        ['ra2016', 'indexed', True], ['dec2016', 'indexed', True]])

Retrieving table 'user_mwidmaie.stellar_masses'
Table 'user_mwidmaie.stellar_masses' updated.


## Joining stellar_masses and stellar_radii

We can join the stellar_masses and stellar_radii tables using the following ADQL query:
```sql
SELECT TOP 150000 stellar_radii.source_id, gaia_ra as ra, gaia_dec as dec, radius, radius_error, mass_primary, mass_secondary, mass_primary_lower, mass_primary_upper
FROM {stellar_radii_table_name} AS stellar_radii
JOIN {stellar_masses_table_name} AS stellar_masses
ON stellar_radii.source_id = stellar_masses.source_id
```

In [26]:
import gzip

stellar_radii_table_name = 'user_' + username + ".stellar_radii"
stellar_masses_table_name = 'user_' + username + ".stellar_masses"
combined_table_name = 'user_' + username + ".stellar_masses_and_radii"

try:
    stellar_mass_and_radii = Gaia.load_table(combined_table_name)
except:
    # Table doesn't exist so we're gonna upload from the local stellar_ldd.vot file
    print("Table does not exist, running queries and uploading from local machine.")
    job = Gaia.launch_job(f"""SELECT TOP 150000 stellar_radii.source_id, gaia_ra as ra, gaia_dec as dec, radius, radius_error, mass_primary, mass_secondary, mass_primary_lower, mass_primary_upper
FROM {stellar_radii_table_name} AS stellar_radii
JOIN {stellar_masses_table_name} AS stellar_masses
ON stellar_radii.source_id = stellar_masses.source_id""", dump_to_file=True, output_format='votable',
                          output_file='data/stellar_masses_and_radii.vot.gz', verbose=True)

    results = job.get_results()

    with gzip.open('data/stellar_masses_and_radii.vot.gz', 'rb') as f:
        with open('data/stellar_masses_and_radii.vot', 'wb') as f2:
            f2.write(f.read())

    table = Table.read('data/stellar_masses_and_radii.vot')
    # Set units for the table
    table['radius'].unit = 'Rsun'
    table['radius_error'].unit = 'Rsun'
    table['mass_primary'].unit = 'Msun'
    table['mass_secondary'].unit = 'Msun'
    table['mass_primary_lower'].unit = 'Msun'
    table['mass_primary_upper'].unit = 'Msun'
    table['ra'].unit = 'deg'
    table['dec'].unit = 'deg'

    # Save as a VOTable and upload to Gaia
    table.write('data/stellar_masses_and_radii.vot', format='votable', overwrite=True)

    Gaia.upload_table(upload_resource='data/stellar_masses_and_radii.vot', table_name='stellar_masses_and_radii',
                      format='votable')
    Gaia.update_user_table(table_name=combined_table_name,
                           list_of_changes=[['ra', 'flags', 'Ra'], ['dec', 'flags', 'Dec'], ['ra', 'indexed', True],
                                            ['dec', 'indexed', True]])

    # Remove the old tables
    Gaia.delete_user_table(table_name=stellar_radii_table_name)
    Gaia.delete_user_table(table_name=stellar_masses_table_name)

Retrieving table 'user_mwidmaie.stellar_masses_and_radii'
500 Error 500:
esavo.tap.TAPException: Code: 404, msg: Table 'user_mwidmaie.stellar_masses_and_radii' not found.
Table does not exist, running queries and uploading from local machine.
Launched query: 'SELECT TOP 150000 stellar_radii.source_id, gaia_ra as ra, gaia_dec as dec, radius, radius_error, mass_primary, mass_secondary, mass_primary_lower, mass_primary_upper
FROM user_mwidmaie.stellar_radii AS stellar_radii
JOIN user_mwidmaie.stellar_masses AS stellar_masses
ON stellar_radii.source_id = stellar_masses.source_id'
------>https
host = gea.esac.esa.int:443
context = /tap-server/tap/sync
Content-type = application/x-www-form-urlencoded
200 200
[('Date', 'Fri, 19 Jan 2024 18:27:19 GMT'), ('Server', 'Apache/2.4.6 (CentOS) OpenSSL/1.0.2k-fips mod_jk/1.2.43'), ('Cache-Control', 'no-cache, no-store, max-age=0, must-revalidate'), ('Pragma', 'no-cache'), ('Expires', '0'), ('X-XSS-Protection', '1; mode=block'), ('X-Frame-Options', 'SA

## Calculating Stellar Core Temperature
We can calculate the stellar core temperature using a generalized formula based on the Lane-Emden equation: (Solved using the Sun's data as a reference)

According to [this paper](https://jila.colorado.edu/~ajsh/courses/astr1120_03/text/chapter5/Hydrostatic.htm), this equation works as a good generalization for all main sequence stars.

The formula is as follows:
![Stellar Core Temperature Formula](imgs/latex_core_temp.png)
Where:
`T sub c` = Stellar core temperature (in Kelvin)
`M sub *` = Stellar mass (in solar masses)
`R sub *` = Stellar radius (in radii)

We will use pandas to calculate the core temperature for each star in the dataset.

In [27]:
import numpy as np

# Load stellar_masses_and_radii.vot as a Pandas DataFrame
combined_table_name = 'user_' + username + ".stellar_masses_and_radii"
combined_table: Table = Table.read('data/stellar_masses_and_radii.vot')
combined_table_df = combined_table.to_pandas(index='source_id')

# Calculate core temperature
combined_table_df['core_temp'] = 1.5e+7 * combined_table_df['mass_primary'] / np.power(combined_table_df['radius'], -1)

## Calculating Stellar Core Temperature Error
Now that we have the core temperature, we must calculate the error. This is a simple procedure relying on error propagation with multiplied values (`M sub *` and `R sub *`)

The formula is as follows:
![Stellar Core Temperature Error Formula](imgs/latex_core_temp_error.png)

Where:
`ΔT sub c` = Absolute stellar core temperature error (in Kelvin)
`T sub c` = Stellar core temperature (in Kelvin)
`ΔM sub *` = Stellar mass error (in solar masses), calculated using the following formula:
![Stellar Mass Error Formula](imgs/latex_mass_error.png)

Where:
`ΔM sub *` = Stellar mass error (in solar masses)
`M sub upper` = Upper stellar mass (in solar masses), Given
`M sub lower` = Lower stellar mass (in solar masses), Given

`ΔR sub *` = Stellar radius error (in solar radii), Given
`M sub avg` = Average of upper and lower stellar masses (in solar masses)
`R sub *` = Stellar radius (in solar radii), Given

We will use pandas to calculate the core temperature error for each star in the dataset.

In [28]:
# Calculate core temperature error
combined_table_df['mass_error'] = np.abs(
    combined_table_df['mass_primary_upper'] - combined_table_df['mass_primary_lower']) / 2
combined_table_df['mass_avg'] = (combined_table_df['mass_primary_upper'] + combined_table_df['mass_primary_lower']) / 2

combined_table_df['core_temp_rel_error'] = np.sqrt(
    np.square(combined_table_df['mass_error'] / combined_table_df['mass_avg']) + np.square(
        -1 * combined_table_df['radius_error']))
combined_table_df['core_temp_abs_error'] = combined_table_df['core_temp'] * combined_table_df['core_temp_rel_error']
combined_table_df.drop(columns=['mass_error', 'mass_avg'], inplace=True)

combined_table_df

,ra,dec,radius,radius_error,mass_primary,mass_secondary,mass_primary_lower,mass_primary_upper,core_temp,mass_error,mass_avg,core_temp_rel_error,core_temp_abs_error
source_id,,,,,,,,,,,,,
1011264588308575232,139.380013,46.817283,2.638851,0.073952,2.064137,1.648135,1.975810,2.161963,8.170424e+07,0.093076,2.068886,0.086561,7.072419e+06
1074883087005896320,176.484167,72.095963,0.941160,0.022193,0.773057,0.097968,0.724040,0.825749,1.091355e+07,0.050855,0.774895,0.069279,7.560783e+05
1137428256197026688,126.465673,77.218546,3.012855,0.102631,1.808754,1.997531,1.716472,1.916902,8.174268e+07,0.100215,1.816687,0.116517,9.524422e+06
1139059897091616512,124.128159,79.500683,1.380779,0.034146,1.074839,NaN,0.977008,1.132453,2.226174e+07,0.077723,1.054730,0.081216,1.808020e+06
1154494768307323776,223.466160,2.775011,1.254070,0.046589,1.252971,0.390569,1.195053,1.303416,2.356969e+07,0.054182,1.249234,0.063653,1.500282e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
831063566054567552,163.460409,46.155726,2.345910,0.089284,1.381726,0.583708,1.305733,1.445168,4.862108e+07,0.069718,1.375450,0.102668,4.991848e+06
842730209053336192,168.799737,54.157511,0.756177,0.021991,0.677829,NaN,0.543633,0.727880,7.688387e+06,0.092124,0.635756,0.146563,1.126833e+06
862138371693741312,170.147868,62.364017,2.097585,0.063009,1.492176,NaN,1.436168,1.548184,4.694950e+07,0.056008,1.492176,0.073341,3.443323e+06


### Save to Gaia
Export to VOT, update labels then push to Gaia

In [40]:
table: Table = Table.from_pandas(combined_table_df, index='source_id')

# Update labels
table['source_id'].description = 'Gaia Source ID'
table['ra'].description = 'Right Ascension'
table['dec'].description = 'Declination'
table['radius'].description = 'Stellar Radius'
table['radius_error'].description = 'Stellar Radius Error'
table['mass_primary'].description = 'Primary Stellar Mass'
table['mass_secondary'].description = 'Secondary Stellar Mass'
table['mass_primary_lower'].description = 'Primary Stellar Mass Lower Bound'
table['mass_primary_upper'].description = 'Primary Stellar Mass Upper Bound'
table['core_temp'].description = 'Stellar Core Temperature'
table['core_temp_abs_error'].description = 'Absolute Stellar Core Temperature Error'
table['core_temp_rel_error'].description = 'Relative Stellar Core Temperature Error'

# Update units
table['ra'].unit = 'deg'
table['dec'].unit = 'deg'
table['radius'].unit = 'Rsun'
table['radius_error'].unit = 'Rsun'
table['mass_primary'].unit = 'Msun'
table['mass_secondary'].unit = 'Msun'
table['mass_primary_lower'].unit = 'Msun'
table['mass_primary_upper'].unit = 'Msun'
table['core_temp'].unit = 'K'
table['core_temp_abs_error'].unit = 'K'

table.meta['description'] = 'Final dataset of Stellar Masses, Radii, and Core Temperatures'

# Save as a VOTable and upload to Gaia
table.write('data/stellar_dataset_final.vot', format='votable', overwrite=True, table_id='stellar_dataset_final')

final_table_name = 'user_' + username + ".stellar_dataset_final"

Gaia.upload_table(upload_resource='data/stellar_dataset_final.vot', table_name='stellar_dataset_final',
                  format='votable')

Gaia.update_user_table(table_name=final_table_name,
                       list_of_changes=[['ra', 'flags', 'Ra'], ['dec', 'flags', 'Dec'], ['ra', 'indexed', True],
                                        ['dec', 'indexed', True], ['source_id', 'indexed', True]])

# Delete the old tables
Gaia.delete_user_table(table_name=combined_table_name)

Sending file: data/stellar_dataset_final.vot
Uploaded table 'stellar_dataset_final'.
Retrieving table 'user_mwidmaie.stellar_dataset_final'
500 Error 500:
Can not modify table 'user_mwidmaie.stellar_dataset_final'. It is being used in a running query.


HTTPError: Error 500:
Can not modify table 'user_mwidmaie.stellar_dataset_final'. It is being used in a running query.